# Ejercicios de Procedimientos y Cursores
**Código de inicialización**

Este código permite inicializar las bibliotecas para usar SQLite en la máquina virtual de Google Colab. El código además elimina cualquier dato almacenado anteriormente en el archvio `ejemplos.db`. Es recomendable volver a correr el código cada vez que se comience con la resolución de un nuevo ejercicio.

In [2]:
!pip install pymysql
%load_ext sql
!rm -rf ejemplos.db
%sql sqlite:///ejemplos.db

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 2.7 MB/s eta 0:00:00


## Ejercicio 1: Actualizar Estados de Órdenes

**Enunciado:**

Tienes una tabla llamada `Ordenes` con la siguiente estructura:

```sql
CREATE TABLE Ordenes (
    Id INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    FechaOrden DATE NOT NULL,
    Estado VARCHAR(20) NOT NULL, -- Ejemplos: 'Pendiente', 'Procesando', 'Enviado', 'Entregado'
    Total DECIMAL(10,2) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ActualizarEstadoOrdenes` que utilice un cursor para actualizar el estado de las órdenes basándose en la fecha de la orden y el estado actual.

**Requisitos:**

1. Declarar un cursor que seleccione `Id`, `FechaOrden` y `Estado` de todas las órdenes que aún no están en estado 'Entregado'.
2. Iterar sobre cada orden y aplicar las siguientes reglas de actualización:
   - Si la orden tiene más de 30 días y está en estado 'Pendiente', actualizar el estado a 'Cancelada'.
   - Si la orden tiene entre 15 y 30 días y está en estado 'Pendiente', actualizar el estado a 'Procesando'.
   - Si la orden tiene entre 1 y 14 días y está en estado 'Procesando', actualizar el estado a 'Enviado'.
3. Asegurar que el procedimiento maneje correctamente la apertura y cierre del cursor.



In [3]:
%%sql
CREATE PROCEDURE ActualizarEstadoOrdenes
AS
BEGIN
    DECLARE @Id INT,
            @FechaOrden DATE,
            @Estado VARCHAR(20);

    DECLARE cursor_ordenes CURSOR FOR
    SELECT Id, FechaOrden, Estado
    FROM Ordenes
    WHERE Estado <> 'Entregado';
    BEGIN TRY
        OPEN cursor_ordenes;
        FETCH NEXT FROM cursor_ordenes INTO @Id, @FechaOrden, @Estado;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            IF DATEDIFF(DAY, @FechaOrden, GETDATE()) > 30 AND @Estado = 'Pendiente'
            BEGIN
                UPDATE Ordenes
                SET Estado = 'Cancelada'
                WHERE Id = @Id;
            END
            ELSE IF DATEDIFF(DAY, @FechaOrden, GETDATE()) BETWEEN 15 AND 30 AND @Estado = 'Pendiente'
            BEGIN
                UPDATE Ordenes
                SET Estado = 'Procesando'
                WHERE Id = @Id;
            END
            ELSE IF DATEDIFF(DAY, @FechaOrden, GETDATE()) BETWEEN 1 AND 14 AND @Estado = 'Procesando'
            BEGIN
                UPDATE Ordenes
                SET Estado = 'Enviado'
                WHERE Id = @Id;
            END

            FETCH NEXT FROM cursor_ordenes INTO @Id, @FechaOrden, @Estado;
            END

        CLOSE cursor_ordenes;
        DEALLOCATE cursor_ordenes;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH
END;



 * sqlite:///ejemplos.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE ActualizarEstadoOrdenes
AS
BEGIN
    DECLARE @Id INT,
            @FechaOrden DATE,
            @Estado VARCHAR(20);

    DECLARE cursor_ordenes CURSOR FOR
    SELECT Id, FechaOrden, Estado
    FROM Ordenes
    WHERE Estado <> 'Entregado';
    BEGIN TRY
        OPEN cursor_ordenes;
        FETCH NEXT FROM cursor_ordenes INTO @Id, @FechaOrden, @Estado;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            IF DATEDIFF(DAY, @FechaOrden, GETDATE()) > 30 AND @Estado = 'Pendiente'
            BEGIN
                UPDATE Ordenes
                SET Estado = 'Cancelada'
                WHERE Id = @Id;
            END
            ELSE IF DATEDIFF(DAY, @FechaOrden, GETDATE()) BETWEEN 15 AND 30 AND @Estado = 'Pendiente'
            BEGIN
                UPDATE Ordenes
                SET Estado = 'Procesando'
                WHERE Id = @Id;
            END
            ELSE IF DATEDIFF(

## Ejercicio 2: Generar Historial de Cambios

**Enunciado:**

Tienes una tabla llamada `Inventario` y otra tabla llamada `HistorialInventario` para registrar cambios en el inventario:

```sql
CREATE TABLE Inventario (
    ProductoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProducto VARCHAR(100) NOT NULL,
    Cantidad INT NOT NULL,
    Precio DECIMAL(10,2) NOT NULL
);

CREATE TABLE HistorialInventario (
    HistorialId INT PRIMARY KEY IDENTITY(1,1),
    ProductoId INT FOREIGN KEY REFERENCES Inventario(ProductoId),
    FechaCambio DATETIME NOT NULL,
    Cambio INT NOT NULL, -- Puede ser positivo o negativo
    Motivo VARCHAR(255) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `RegistrarCambiosInventario` que utilice un cursor para procesar una lista de cambios en el inventario y registrar cada cambio en la tabla `HistorialInventario`.

**Requisitos:**

1. El procedimiento debe recibir como parámetro una tabla temporal o una variable de tabla que contenga los cambios a procesar, con las columnas: `ProductoId`, `Cambio`, `Motivo`.
2. Declarar un cursor que seleccione cada registro de la lista de cambios.
3. Para cada cambio:
   - Actualizar la cantidad del producto en la tabla `Inventario`.
   - Insertar un registro en la tabla `HistorialInventario` con la fecha actual, el cambio aplicado y el motivo.
4. Implementar manejo de transacciones para asegurar que todos los cambios se apliquen correctamente o se reviertan en caso de error.
5. Manejar adecuadamente la apertura y cierre del cursor, así como el manejo de posibles excepciones.



In [4]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE RegistrarCambiosInventario
    @ListaCambios TABLE (
        ProductoId INT,
        Cambio INT,
        Motivo VARCHAR(255)
    )
AS
BEGIN
    DECLARE @ProductoId INT,
            @Cambio INT,
            @Motivo VARCHAR(255);

    DECLARE cursor_cambios CURSOR FOR
    SELECT ProductoId, Cambio, Motivo
    FROM @ListaCambios;
    BEGIN TRY
        BEGIN TRANSACTION;

        OPEN cursor_cambios;
        FETCH NEXT FROM cursor_cambios INTO @ProductoId, @Cambio, @Motivo;

        WHILE @@FETCH_STATUS = 0
        BEGIN

            UPDATE
            WHERE @ProductoId;


            INSERT INTO HistorialInventario (ProductoId, FechaCambio, Cambio, Motivo)
            VALUES (@ProductoId, GETDATE(), @Cambio, @Motivo);
            FETCH NEXT FROM cursor_cambios INTO @ProductoId, @Cambio, @Motivo;
            END

        CLOSE cursor_cambios;
        DEALLOCATE cursor_cambios;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;



 * sqlite:///ejemplos.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE RegistrarCambiosInventario
    @ListaCambios TABLE (
        ProductoId INT,
        Cambio INT,
        Motivo VARCHAR(255)
    )
AS
BEGIN
    DECLARE @ProductoId INT,
            @Cambio INT,
            @Motivo VARCHAR(255);

    DECLARE cursor_cambios CURSOR FOR
    SELECT ProductoId, Cambio, Motivo
    FROM @ListaCambios;
    BEGIN TRY
        BEGIN TRANSACTION;

        OPEN cursor_cambios;
        FETCH NEXT FROM cursor_cambios INTO @ProductoId, @Cambio, @Motivo;

        WHILE @@FETCH_STATUS = 0
        BEGIN

            UPDATE
            WHERE @ProductoId;


            INSERT INTO HistorialInventario (ProductoId, FechaCambio, Cambio, Motivo)
            VALUES (@ProductoId, GETDATE(), @Cambio, @Motivo);
            FETCH NEXT FROM cursor_cambios INTO @ProductoId, @Cambio, @Motivo;
            END

        CLOSE cursor_cambios;
        DEALLOCATE cursor_cambios;

       

## Ejercicio 3: Validación de Datos

**Enunciado:**

Tienes una tabla llamada `Usuarios` con la siguiente estructura:

```sql
CREATE TABLE Usuarios (
    Id INT PRIMARY KEY IDENTITY(1,1),
    NombreUsuario VARCHAR(50) NOT NULL,
    Email VARCHAR(100) NOT NULL,
    FechaRegistro DATE NOT NULL,
    Estado VARCHAR(20) NOT NULL -- Ejemplos: 'Activo', 'Inactivo', 'Pendiente'
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ValidarUsuarios` que utilice un cursor para revisar cada usuario y actualizar su estado basado en criterios específicos.

**Requisitos:**

1. Declarar un cursor que seleccione `Id`, `FechaRegistro` y `Estado` de todos los usuarios.
2. Iterar sobre cada usuario y aplicar las siguientes reglas:
   - Si el usuario ha estado en estado 'Pendiente' por más de 7 días, actualizar su estado a 'Inactivo'.
   - Si el usuario está en estado 'Activo' y su fecha de registro fue hace más de un año, enviar una notificación (simulado con un comentario en el código).
3. Asegurar el manejo correcto del cursor y posibles errores durante el proceso.



In [5]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE ValidarUsuarios
AS
BEGIN
    DECLARE @Id INT,
            @FechaRegistro DATE,
            @Estado VARCHAR(20);

    DECLARE cursor_usuarios CURSOR FOR
    SELECT Id, FechaRegistro, Estado
    FROM Usuarios;

    BEGIN TRY
        OPEN cursor_usuarios;
        FETCH NEXT FROM cursor_usuarios INTO @Id, @FechaRegistro, @Estado;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            IF @Estado = 'Pendiente' AND DATEDIFF(DAY, @FechaRegistro, GETDATE()) > 7
            BEGIN
                UPDATE Usuarios
                SET Estado = 'Inactivo'
                WHERE Id = @Id;
            END
            ELSE IF @Estado = 'Activo' AND DATEDIFF(YEAR, @FechaRegistro, GETDATE()) >
            BEGIN
                -- Simulando envío de notificación
                PRINT 'Notificación enviada para el usuario con ID ' + CAST(@Id AS VARCHAR(10));

            END

            FETCH NEXT FROM cursor_usuarios INTO @Id, @FechaRegistro, @Estado;
        END

        CLOSE cursor_usuarios;
        DEALLOCATE cursor_usuarios;
    END TRY
    BEGIN CATCH
        THROW;
    END CATCH
END;

 * sqlite:///ejemplos.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE ValidarUsuarios
AS
BEGIN
    DECLARE @Id INT,
            @FechaRegistro DATE,
            @Estado VARCHAR(20);

    DECLARE cursor_usuarios CURSOR FOR
    SELECT Id, FechaRegistro, Estado
    FROM Usuarios;

    BEGIN TRY
        OPEN cursor_usuarios;
        FETCH NEXT FROM cursor_usuarios INTO @Id, @FechaRegistro, @Estado;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            IF @Estado = 'Pendiente' AND DATEDIFF(DAY, @FechaRegistro, GETDATE()) > 7
            BEGIN
                UPDATE Usuarios
                SET Estado = 'Inactivo'
                WHERE Id = @Id;
            END
            ELSE IF @Estado = 'Activo' AND DATEDIFF(YEAR, @FechaRegistro, GETDATE()) >
            BEGIN
                -- Simulando envío de notificación
                PRINT 'Notificación enviada para el usuario con ID ' + CAST(@Id AS VARCHAR(10));

            END

            FETCH NEXT 

## Ejercicio 4: Asignación de Tareas

**Enunciado:**

Tienes dos tablas: `Empleados` y `TareasAsignadas`:

```sql
CREATE TABLE Empleados (
    Id INT PRIMARY KEY IDENTITY(1,1),
    Nombre VARCHAR(100) NOT NULL,
    Departamento VARCHAR(50) NOT NULL,
    TareasPendientes INT DEFAULT 0
);

CREATE TABLE Tareas (
    TareaId INT PRIMARY KEY IDENTITY(1,1),
    Descripcion VARCHAR(255) NOT NULL,
    Prioridad INT NOT NULL -- 1: Alta, 2: Media, 3: Baja
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `AsignarTareas` que utilice un cursor para asignar tareas a empleados basándose en la cantidad de tareas pendientes y la prioridad de las tareas.

**Requisitos:**

1. Declarar un cursor que seleccione todas las tareas ordenadas por prioridad (de alta a baja).
2. Para cada tarea, buscar al empleado con la menor cantidad de tareas pendientes en el mismo departamento (puedes asumir un departamento específico o agregar uno en las tablas).
3. Asignar la tarea al empleado seleccionado:
   - Insertar un registro en la tabla `TareasAsignadas` (debes crear esta tabla con las columnas `AsignacionId`, `EmpleadoId`, `TareaId`, `FechaAsignacion`).
   - Incrementar el contador de `TareasPendientes` del empleado en la tabla `Empleados`.
4. Implementar manejo de transacciones para asegurar la consistencia de los datos.
5. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores.


In [6]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE AsignarTareas
AS
BEGIN
    DECLARE @TareaId INT,
            @Descripcion VARCHAR(255),
            @Prioridad INT;

    DECLARE cursor_tareas CURSOR FOR
    SELECT TareaId, Descripcion, Prioridad
    FROM Tareas
    ORDER BY Prioridad DESC;

    BEGIN TRY
        BEGIN TRANSACTION;

        OPEN cursor_tareas;
        FETCH NEXT FROM cursor_tareas INTO @TareaId, @Descripcion, @Prioridad;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            -- Buscar al empleado con la menor cantidad de tareas pendientes en el mismo departamento
            DECLARE @EmpleadoId INT;
            SELECT TOP 1 @EmpleadoId = Id
            FROM Empleados
            WHERE Departamento = (SELECT Departamento FROM Empleados WHERE Id = @EmpleadoId)
            ORDER BY TareasPendientes;

            -- Asignar la tarea al empleado
            INSERT INTO TareasAsignadas (EmpleadoId, TareaId, FechaAsignacion)
            VALUES (@EmpleadoId, @TareaId, GETDATE());

            -- Incrementar el contador de TareasPendientes del empleado
            UPDATE
            WHERE
            FETCH NEXT FROM cursor_tareas INTO @TareaId, @Descripcion, @Prioridad;
            END

        CLOSE cursor_tareas;
        DEALLOCATE cursor_tareas;

        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH
END;

 * sqlite:///ejemplos.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE AsignarTareas
AS
BEGIN
    DECLARE @TareaId INT,
            @Descripcion VARCHAR(255),
            @Prioridad INT;

    DECLARE cursor_tareas CURSOR FOR
    SELECT TareaId, Descripcion, Prioridad
    FROM Tareas
    ORDER BY Prioridad DESC;

    BEGIN TRY
        BEGIN TRANSACTION;

        OPEN cursor_tareas;
        FETCH NEXT FROM cursor_tareas INTO @TareaId, @Descripcion, @Prioridad;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            -- Buscar al empleado con la menor cantidad de tareas pendientes en el mismo departamento
            DECLARE @EmpleadoId INT;
            SELECT TOP 1 @EmpleadoId = Id
            FROM Empleados
            WHERE Departamento = (SELECT Departamento FROM Empleados WHERE Id = @EmpleadoId)
            ORDER BY TareasPendientes;

            -- Asignar la tarea al empleado
            INSERT INTO TareasAsignadas (EmpleadoId, TareaId, FechaAs

## Ejercicio 5: Cálculo de Bonificaciones

**Enunciado:**

Tienes una tabla llamada `Ventas` y otra tabla llamada `Empleados`:

```sql
CREATE TABLE Empleados (
    Id INT PRIMARY KEY IDENTITY(1,1),
    Nombre VARCHAR(100) NOT NULL,
    Departamento VARCHAR(50) NOT NULL,
    Salario DECIMAL(10,2) NOT NULL
);

CREATE TABLE Ventas (
    VentaId INT PRIMARY KEY IDENTITY(1,1),
    EmpleadoId INT FOREIGN KEY REFERENCES Empleados(Id),
    Monto DECIMAL(10,2) NOT NULL,
    FechaVenta DATE NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `CalcularBonificaciones` que utilice un cursor para calcular y asignar bonificaciones a los empleados basadas en sus ventas mensuales.

**Requisitos:**

1. Declarar un cursor que seleccione cada `EmpleadoId` y la suma total de sus ventas en el último mes.
2. Para cada empleado:
   - Si las ventas totales exceden un umbral definido (por ejemplo, $10,000), calcular una bonificación del 10% sobre las ventas.
   - Actualizar el salario del empleado sumando la bonificación calculada.
   - Insertar un registro en una tabla `Bonificaciones` (debes crear esta tabla con las columnas `BonificacionId`, `EmpleadoId`, `MontoBonificacion`, `FechaBonificacion`).
3. Implementar manejo de transacciones para asegurar que las actualizaciones y las inserciones se realicen correctamente.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso.



In [7]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE CalcularBonificaciones
AS
BEGIN
      DECLARE @EmpleadoId INT,
              @VentasTotales DECIMAL(10,2);

      DECLARE cursor_ventas CURSOR FOR
      SELECT EmpleadoId, SUM(Monto)
      FROM Ventas
      WHERE FechaVenta >= DATEADD(MONTH, -1, GETDATE())
      GROUP BY EmpleadoId;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_ventas;
        FETCH NEXT FROM cursor_ventas INTO @EmpleadoId, @VentasTotales;
        WHILE @@FETCH_STATUS
        BEGIN
          IF @VentasTotales > 10000
          BEGIN
            DECLARE @Bonificacion DECIMAL(10,2) = @VentasTotales * 0.1;
            UPDATE Empleados
            SET Salario = Salario + @Bonificacion
            WHERE Id = @EmpleadoId;
            INSERT INTO Bonificaciones (EmpleadoId, MontoBonificacion, FechaBonificacion)
            VALUES (@EmpleadoId, @Bonificacion, GETDATE());
          END
          FETCH NEXT FROM cursor_ventas INTO @EmpleadoId, @VentasTotales;
          END
        CLOSE cursor_ventas;
        DEALLOCATE cursor_ventas;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;



 * sqlite:///ejemplos.db
(sqlite3.OperationalError) near "PROCEDURE": syntax error
[SQL: CREATE PROCEDURE CalcularBonificaciones
AS
BEGIN
      DECLARE @EmpleadoId INT,
              @VentasTotales DECIMAL(10,2);

      DECLARE cursor_ventas CURSOR FOR
      SELECT EmpleadoId, SUM(Monto)
      FROM Ventas
      WHERE FechaVenta >= DATEADD(MONTH, -1, GETDATE())
      GROUP BY EmpleadoId;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_ventas;
        FETCH NEXT FROM cursor_ventas INTO @EmpleadoId, @VentasTotales;
        WHILE @@FETCH_STATUS
        BEGIN
          IF @VentasTotales > 10000
          BEGIN
            DECLARE @Bonificacion DECIMAL(10,2) = @VentasTotales * 0.1;
            UPDATE Empleados
            SET Salario = Salario + @Bonificacion
            WHERE Id = @EmpleadoId;
            INSERT INTO Bonificaciones (EmpleadoId, MontoBonificacion, FechaBonificacion)
            VALUES (@EmpleadoId, @Bonificacion, GETDATE());
          END
          FETCH NEXT 

## Ejercicio 6: Migración de Datos

**Enunciado:**

Tienes una tabla antigua `ClientesAntiguos` y una nueva tabla `ClientesActuales` con estructuras ligeramente diferentes:

```sql
CREATE TABLE ClientesAntiguos (
    ClienteId INT PRIMARY KEY IDENTITY(1,1),
    NombreCompleto VARCHAR(150) NOT NULL,
    Telefono VARCHAR(20),
    Direccion VARCHAR(255),
    FechaRegistro DATETIME NOT NULL
);

CREATE TABLE ClientesActuales (
    ClienteId INT PRIMARY KEY IDENTITY(1,1),
    Nombre VARCHAR(100) NOT NULL,
    Apellido VARCHAR(50) NOT NULL,
    Telefono VARCHAR(20),
    Direccion VARCHAR(255),
    FechaRegistro DATE NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `MigrarClientes` que utilice un cursor para migrar datos de la tabla `ClientesAntiguos` a `ClientesActuales`, dividiendo el campo `NombreCompleto` en `Nombre` y `Apellido`.

**Requisitos:**

1. Declarar un cursor que seleccione todos los registros de la tabla `ClientesAntiguos`.
2. Para cada registro:
   - Dividir `NombreCompleto` en `Nombre` y `Apellido` (puedes asumir que el apellido es la última palabra del nombre completo).
   - Insertar un nuevo registro en la tabla `ClientesActuales` con los datos transformados.
3. Implementar manejo de transacciones para asegurar que todos los registros se migren correctamente o se reviertan en caso de error.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles excepciones durante la migración.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE MigrarClientes
AS
BEGIN
      DECLARE
            @NombreCompleto VARCHAR(150),
            @Nombre VARCHAR(100),
            @Apellido VARCHAR(50);

      DECLARE cursor_clientes CURSOR FOR
      SELECT NombreCompleto
      FROM ClientesAntiguos;

      BEGIN TRY
        BEGIN TRANSACTION;

        OPEN cursor_clientes;
        FETCH NEXT FROM cursor_clientes INTO @NombreCompleto;

        WHILE @@FETCH_STATUS = 0
        BEGIN
            SET @Nombre = SUBSTRING(@NombreCompleto, 1, CHARINDEX(' ', @NombreCompleto)
            SET @Apellido = SUBSTRING(@NombreCompleto, CHARINDEX(' ', @NombreCompleto) + 1, LEN(@NombreCompleto))
            INSERT INTO ClientesActuales (Nombre, Apellido, FechaRegistro)
            VALUES
            FETCH NEXT FROM cursor_clientes INTO @NombreCompleto;
        END

        CLOSE cursor_clientes;
        DEALLOCATE cursor_clientes;

        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;











## Ejercicio 7: Consolidación de Datos

**Enunciado:**

Tienes una tabla `Pedidos` con la siguiente estructura:

```sql
CREATE TABLE Pedidos (
    PedidoId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    ProductoId INT NOT NULL,
    Cantidad INT NOT NULL,
    PrecioUnitario DECIMAL(10,2) NOT NULL,
    FechaPedido DATE NOT NULL
);
```

Y otra tabla `ConsolidadoVentas` para almacenar las ventas consolidadas por cliente y producto:

```sql
CREATE TABLE ConsolidadoVentas (
    ConsolidadoId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    ProductoId INT NOT NULL,
    CantidadTotal INT NOT NULL,
    IngresoTotal DECIMAL(15,2) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ConsolidarVentas` que utilice un cursor para procesar todos los pedidos y actualizar la tabla `ConsolidadoVentas` con la suma total de cantidad e ingresos por cliente y producto.

**Requisitos:**

1. Declarar un cursor que seleccione `ClienteId`, `ProductoId`, `Cantidad` y `PrecioUnitario` de la tabla `Pedidos`.
2. Para cada pedido:
   - Verificar si ya existe un registro en `ConsolidadoVentas` para el par `ClienteId` y `ProductoId`.
   - Si existe, actualizar `CantidadTotal` y `IngresoTotal` sumando los valores del pedido.
   - Si no existe, insertar un nuevo registro con los valores del pedido.
3. Implementar manejo de transacciones para asegurar la consistencia de los datos durante la consolidación.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
 CREATE PROCEDURE ConsolidarVentas
 AS
 BEGIN
      DECLARE @ClienteId INT,
            @ProductoId INT,
            @Cantidad INT,
            @PrecioUnitario DECIMAL(10,2);

      DECLARE cursore_pedidos CURSOR FOR
      SELECT ClienteId, ProductoId, Cantidad, PrecioUnitario;
      FROM Pedidos;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursore_pedidos;
        FETCH_NEXT FROM cursore_pedidos INTO @ClienteId, @ProductoId, @Cantidad, @PrecioUnitario;
        WHILE @@FETCH_STATUS
        BEGIN
          IF EXISTS (SELECT 1 FROM ConsolidadoVentas WHERE ClienteId = @ClienteId AND ProductoId = @ProductoId)
          BEGIN
            UPDATE ConsolidadoVentas
            SET CantidadTotal = CantidadTotal + @Cantidad,
            IngresoTotal = IngresoTotal + (@Cantidad * @PrecioUnitario)
            WHERE ClienteId = @ClienteId AND ProductoId = @ProductoId;
            END
            ELSE
            BEGIN
              INSERT INTO ConsolidadoVentas (ClienteId, ProductoId, CantidadTotal, IngresoTotal)
              VALUES (@ClienteId, @ProductoId, @Cantidad, @PrecioUnitario * @Cantidad);
            END
            FETCH NEXT FROM cursore_pedidos INTO @ClienteId, @ProductoId, @Cantidad, @PrecioUnitario;
        END


## Ejercicio 8: Sincronización de Bases de Datos

**Enunciado:**

Tienes dos bases de datos diferentes: una base de datos `Local` y una base de datos `Remota`. Ambas tienen una tabla `Inventario`, pero la base de datos remota necesita estar sincronizada con la local.

```sql
-- En Base de Datos Local
CREATE TABLE InventarioLocal (
    ProductoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProducto VARCHAR(100) NOT NULL,
    Cantidad INT NOT NULL,
    Precio DECIMAL(10,2) NOT NULL,
    UltimaActualizacion DATETIME NOT NULL
);

-- En Base de Datos Remota
CREATE TABLE InventarioRemoto (
    ProductoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProducto VARCHAR(100) NOT NULL,
    Cantidad INT NOT NULL,
    Precio DECIMAL(10,2) NOT NULL,
    UltimaActualizacion DATETIME NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `SincronizarInventario` en la base de datos local que utilice un cursor para comparar y sincronizar los registros de `InventarioLocal` con `InventarioRemoto`.

**Requisitos:**

1. Declarar un cursor que seleccione todos los registros de `InventarioLocal`.
2. Para cada registro:
   - Verificar si existe un registro correspondiente en `InventarioRemoto` basado en `ProductoId`.
   - Si existe y la `UltimaActualizacion` en local es más reciente, actualizar el registro en `InventarioRemoto`.
   - Si no existe, insertar el registro en `InventarioRemoto`.
3. Implementar manejo de transacciones para asegurar que todas las actualizaciones se realicen correctamente.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante la sincronización.
5. (Opcional) Registrar las operaciones realizadas en una tabla de logs para auditoría.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql

## Ejercicio 9: Generación de Facturas

**Enunciado:**

Tienes dos tablas: `Pedidos` y `Facturas`:

```sql
CREATE TABLE Pedidos (
    PedidoId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    FechaPedido DATE NOT NULL,
    TotalPedido DECIMAL(10,2) NOT NULL,
    Facturado BIT DEFAULT 0
);

CREATE TABLE Facturas (
    FacturaId INT PRIMARY KEY IDENTITY(1,1),
    PedidoId INT FOREIGN KEY REFERENCES Pedidos(PedidoId),
    FechaFactura DATE NOT NULL,
    MontoFacturado DECIMAL(10,2) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `GenerarFacturas` que utilice un cursor para procesar todos los pedidos no facturados y generar las facturas correspondientes.

**Requisitos:**

1. Declarar un cursor que seleccione `PedidoId`, `ClienteId`, `TotalPedido` de todos los pedidos donde `Facturado = 0`.
2. Para cada pedido:
   - Insertar un nuevo registro en la tabla `Facturas` con la `FechaFactura` como la fecha actual y el `MontoFacturado` igual al `TotalPedido`.
   - Actualizar el campo `Facturado` del pedido a `1` para indicar que ya ha sido facturado.
3. Implementar manejo de transacciones para asegurar que la inserción en `Facturas` y la actualización en `Pedidos` se realicen de forma atómica.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso de facturación.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE GenerarFacturas
AS
BEGIN
      DECLARE @PedidoId INT,
              @ClienteId INT,
              @TotalPedido DECIMAL(10,2);
      DECLARE cursor_pedidos CURSOR FOR
      SELECT PedidoId, ClienteId, TotalPedido
      FROM Pedidos
      WHERE Facturado = 0;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_pedidos;
        FETCH NEXT FROM cursor_pedidos INTO @PedidoId, @ClienteId, @TotalPedido;
        WHILE @@FETCH_STATUS
        BEGIN
          INSERT INTO Facturas (PedidoId, FechaFactura, MontoFacturado)
          VALUES (@PedidoId, GETDATE(), @TotalPedido);
          UPDATE Pedidos
          SET Facturado = 1
          WHERE PedidoId = @PedidoId;
          FETCH NEXT FROM cursor_pedidos INTO @PedidoId, @ClienteId, @TotalPedido;
        END
        CLOSE cursor_pedidos;
        DEALLOCATE cursor_pedidos;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
        END CATCH
END;

## Ejercicio 10: Actualización de Precios Basada en Competencia

**Enunciado:**

Tienes una tabla `Productos` y una tabla `CompetenciaPrecios` que contiene los precios de productos similares ofrecidos por la competencia:

```sql
CREATE TABLE Productos (
    ProductoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProducto VARCHAR(100) NOT NULL,
    Precio DECIMAL(10,2) NOT NULL,
    Categoria VARCHAR(50) NOT NULL
);

CREATE TABLE CompetenciaPrecios (
    CompetenciaId INT PRIMARY KEY IDENTITY(1,1),
    NombreProducto VARCHAR(100) NOT NULL,
    PrecioCompetencia DECIMAL(10,2) NOT NULL,
    Categoria VARCHAR(50) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `AjustarPreciosCompetitivos` que utilice un cursor para comparar los precios de los productos propios con los de la competencia y ajustar los precios propios en consecuencia.

**Requisitos:**

1. Declarar un cursor que seleccione `ProductoId`, `Precio`, y `Categoria` de la tabla `Productos`.
2. Para cada producto:
   - Buscar el precio promedio de la competencia en la misma categoría desde la tabla `CompetenciaPrecios`.
   - Si el precio propio es mayor en más de un 5% que el promedio de la competencia, reducir el precio propio en un 5%.
   - Si el precio propio es menor en más de un 5%, incrementar el precio propio en un 5%.
   - Si la diferencia es menor o igual al 5%, mantener el precio sin cambios.
3. Implementar manejo de transacciones para asegurar la consistencia de las actualizaciones.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso de ajuste de precios.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE AjustarPreciosCompetitivos
AS
BEGIN
      DECLARE @ProductoId INT,
              @Precio DECIMAL(10,2),
              @Categoria VARCHAR(50),
              @PrecioCompetencia DECIMAL(10,2);

      DECLARE cursor_productos CURSOR FOR
      SELECT ProductoId, Precio, Categoria
      FROM Productos;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_productos;
        FETCH NEXT FROM cursor_productos INTO @ProductoId, @Precio, @Categoria;
        WHILE @@FETCH_STATUS = 0
        BEGIN
          SELECT @PrecioCompetencia = AVG(PrecioCompetencia)
          FROM CompetenciaPrecios
          WHERE Categoria = @Categoria;

          IF @Precio > @PrecioCompetencia * 1.05
          BEGIN
            SET @Precio = @Precio * 0.95;
          END
          ELSE IF @Precio < @PrecioCompetencia * 0.95
          BEGIN
            SET @Precio = @Precio * 1.05;
          END
          UPDATE Productos
          SET Precio = @Precio
          WHERE ProductoId = @ProductoId;
          FETCH NEXT FROM cursor_productos INTO @ProductoId, @Precio, @Categoria;
        END
        CLOSE cursor_productos;
        DEALLOCATE cursor_productos;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;


## Ejercicio 11: Generación de Resúmenes de Producción

**Enunciado:**

Tienes una tabla `Produccion` que registra la producción diaria de diferentes líneas de productos:

```sql
CREATE TABLE Produccion (
    ProduccionId INT PRIMARY KEY IDENTITY(1,1),
    LineaProduccion VARCHAR(50) NOT NULL,
    FechaProduccion DATE NOT NULL,
    CantidadProducida INT NOT NULL
);

CREATE TABLE ResumenProduccion (
    ResumenId INT PRIMARY KEY IDENTITY(1,1),
    LineaProduccion VARCHAR(50) NOT NULL,
    FechaProduccion DATE NOT NULL,
    TotalProducido INT NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `GenerarResumenDiario` que utilice un cursor para generar un resumen diario de producción por línea de producción.

**Requisitos:**

1. Declarar un cursor que seleccione todas las combinaciones únicas de `LineaProduccion` y `FechaProduccion` de la tabla `Produccion`.
2. Para cada combinación:
   - Calcular la suma total de `CantidadProducida`.
   - Insertar un registro en la tabla `ResumenProduccion` con los valores calculados.
3. Asegurar que el procedimiento limpie la tabla `ResumenProduccion` antes de generar el nuevo resumen para evitar duplicados.
4. Implementar manejo de transacciones para asegurar que todas las inserciones se realicen correctamente.
5. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante la generación del resumen.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE GenerarResumenDiario
AS
BEGIN
       DECLARE @LineaProduccion VARCHAR(50),
              @FechaProduccion DATE,
              @TotalProducido INT;

      DECLARE cursor_produccion CURSOR FOR
      SELECT DISTINCT LineaProduccion, FechaProduccion
      FROM Produccion;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_produccion;
        FETCH NEXT FROM cursor_produccion INTO @LineaProduccion, @FechaProduccion;
        WHILE @@FETCH_STATUS
        BEGIN
          SELECT @TotalProducido = SUM(CantidadProducida)
          FROM Produccion
          WHERE LineaProduccion = @LineaProduccion AND FechaProduccion = @FechaProduccion;
          INSERT INTO ResumenProduccion (LineaProduccion, FechaProduccion, TotalProducido)
          VALUES (@LineaProduccion, @FechaProduccion, @TotalProducido);
          FETCH NEXT FROM cursor_produccion INTO @LineaProduccion, @FechaProduccion;
          END
        CLOSE cursor_produccion;
        DEALLOCATE cursor_produccion;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;



## Ejercicio 12: Actualización de Inventario Basada en Devoluciones

**Enunciado:**

Tienes una tabla `Devoluciones` que registra las devoluciones de productos y una tabla `Inventario`:

```sql
CREATE TABLE Devoluciones (
    DevolucionId INT PRIMARY KEY IDENTITY(1,1),
    ProductoId INT NOT NULL,
    CantidadDevuelta INT NOT NULL,
    FechaDevolucion DATE NOT NULL
);

CREATE TABLE Inventario (
    ProductoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProducto VARCHAR(100) NOT NULL,
    Cantidad INT NOT NULL,
    Precio DECIMAL(10,2) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ProcesarDevoluciones` que utilice un cursor para procesar todas las devoluciones y actualizar el inventario en consecuencia.

**Requisitos:**

1. Declarar un cursor que seleccione `ProductoId` y `CantidadDevuelta` de la tabla `Devoluciones`.
2. Para cada devolución:
   - Actualizar la cantidad en la tabla `Inventario` incrementando la `Cantidad` por la `CantidadDevuelta`.
   - (Opcional) Registrar el procesamiento de la devolución en una tabla `HistorialDevoluciones` con detalles de la operación.
3. Implementar manejo de transacciones para asegurar que las actualizaciones se realicen correctamente.
4. Después de procesar una devolución, marcarla como procesada o eliminarla para evitar re-procesamientos.
5. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el procesamiento.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql

## Ejercicio 13: Validación de Integridad Referencial

**Enunciado:**

Tienes dos tablas relacionadas: `Ordenes` y `DetalleOrden`:

```sql
CREATE TABLE Ordenes (
    OrdenId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    FechaOrden DATE NOT NULL,
    TotalOrden DECIMAL(10,2) NOT NULL
);

CREATE TABLE DetalleOrden (
    DetalleId INT PRIMARY KEY IDENTITY(1,1),
    OrdenId INT FOREIGN KEY REFERENCES Ordenes(OrdenId),
    ProductoId INT NOT NULL,
    Cantidad INT NOT NULL,
    PrecioUnitario DECIMAL(10,2) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ValidarIntegridadOrdenes` que utilice un cursor para verificar que el `TotalOrden` en la tabla `Ordenes` coincide con la suma de los `PrecioUnitario * Cantidad` en la tabla `DetalleOrden` para cada orden.

**Requisitos:**

1. Declarar un cursor que seleccione `OrdenId` y `TotalOrden` de la tabla `Ordenes`.
2. Para cada orden:
   - Calcular la suma de `PrecioUnitario * Cantidad` desde la tabla `DetalleOrden` correspondiente.
   - Comparar la suma calculada con el `TotalOrden`.
   - Si hay una discrepancia, registrar la orden en una tabla `OrdenesInconsistentes` con detalles de la discrepancia.
3. Implementar manejo de transacciones si es necesario (por ejemplo, para actualizaciones o registros de inconsistencias).
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante la validación.
5. (Opcional) Enviar una notificación o alerta para las órdenes inconsistentes encontradas.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE ValidarIntegridadOrdenes
AS
BEGIN
      DECLARE @OrdenId INT,
              @TotalOrden DECIMAL(10,2),
              @SumaDetalleOrden DECIMAL(10,2);

      DECLARE cursor_ordenes CURSOR FOR
      SELECT OrdenId, TotalOrden
      FROM Ordenes;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_ordenes;
        FETCH NEXT FROM cursor_ordenes INTO @OrdenId, @TotalOrden;
        WHILE @@FETCH_STATUS
        BEGIN
          SELECT @SumaDetalleOrden = SUM(PrecioUnitario * Cantidad)
          FROM DetalleOrden
          WHERE OrdenId = @OrdenId;
          IF @SumaDetalleOrden
          BEGIN
            INSERT INTO OrdenesInconsistentes (OrdenId, TotalOrden, SumaDetalleOrden)
            VALUES (@OrdenId, @TotalOrden, @SumaDetalleOrden);
          END
          FETCH NEXT FROM cursor_ordenes INTO @OrdenId, @TotalOrden;
        END
        CLOSE cursor_ordenes;
        DEALLOCATE cursor_ordenes;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
      END;

## Ejercicio 14: Asignación de Recursos en Proyectos

**Enunciado:**

Tienes dos tablas: `Proyectos` y `RecursosDisponibles`:

```sql
CREATE TABLE Proyectos (
    ProyectoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProyecto VARCHAR(100) NOT NULL,
    RecursosNecesarios INT NOT NULL,
    RecursosAsignados INT DEFAULT 0,
    Estado VARCHAR(20) NOT NULL -- Ejemplos: 'Pendiente', 'En Curso', 'Completado'
);

CREATE TABLE RecursosDisponibles (
    RecursoId INT PRIMARY KEY IDENTITY(1,1),
    TipoRecurso VARCHAR(50) NOT NULL,
    CantidadDisponible INT NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `AsignarRecursosAProyectos` que utilice un cursor para asignar recursos disponibles a los proyectos en estado 'Pendiente'.

**Requisitos:**

1. Declarar un cursor que seleccione todos los proyectos en estado 'Pendiente', ordenados por `RecursosNecesarios` de mayor a menor.
2. Para cada proyecto:
   - Verificar si hay suficientes recursos disponibles en `RecursosDisponibles`.
   - Si hay suficientes, asignar los recursos al proyecto:
     - Incrementar `RecursosAsignados` en la tabla `Proyectos`.
     - Decrementar `CantidadDisponible` en la tabla `RecursosDisponibles`.
     - Actualizar el estado del proyecto a 'En Curso'.
   - Si no hay suficientes recursos, dejar el proyecto en estado 'Pendiente'.
3. Implementar manejo de transacciones para asegurar que las asignaciones sean consistentes.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso de asignación.

In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE AsignarRecursosAProyectos
AS
BEGIN
      DECLARE @ProyectoId INT,
              @NombreProyecto VARCHAR(100),
              @RecursosNecesarios INT,
              @RecursosAsignados INT,
              @Estado VARCHAR(20),
              @TipoRecurso VARCHAR(50),
              @CantidadDisponible INT;
              DECLARE cursor_proyectos CURSOR FOR
      SELECT ProyectoId, NombreProyecto, RecursosNecesarios, RecursosAsignados, Estado
      FROM Proyectos
      WHERE Estado = 'Pendiente'
      ORDER BY RecursosNecesarios DESC;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_proyectos;
        FETCH NEXT FROM cursor_proyectos INTO @ProyectoId, @NombreProyecto, @RecursosNecesarios, @RecursosAsignados, @Estado;
        WHILE @@FETCH_STATUS
        BEGIN
          SELECT @TipoRecurso = TipoRecurso, @CantidadDisponible = CantidadDisponible
          FROM RecursosDisponibles
          WHERE CantidadDisponible >= @RecursosNecesarios
          IF @CantidadDisponible >= @RecursosNecesarios
          BEGIN
            UPDATE Proyectos
            SET RecursosAsignados = @RecursosAsignados + @RecursosNecesarios, Estado = 'En Curso'
            WHERE ProyectoId = @ProyectoId;
            UPDATE RecursosDisponibles
            SET CantidadDisponible = @CantidadDisponible - @RecursosNecesarios
            WHERE TipoRecurso = @TipoRecurso;
          END
          FETCH NEXT FROM cursor_proyectos INTO @ProyectoId, @NombreProyecto, @RecursosNecesarios, @RecursosAsignados, @Estado;
        END
        CLOSE cursor_proyectos;
        DEALLOCATE cursor_proyectos;
        COMMIT TRANSACTION;
        END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;

## Ejercicio 15: Auditoría de Accesos

**Enunciado:**

Tienes una tabla `Accesos` que registra los accesos de usuarios al sistema:

```sql
CREATE TABLE Accesos (
    AccesoId INT PRIMARY KEY IDENTITY(1,1),
    UsuarioId INT NOT NULL,
    FechaAcceso DATETIME NOT NULL,
    DireccionIP VARCHAR(45) NOT NULL
);

CREATE TABLE AuditoriaAccesos (
    AuditoriaId INT PRIMARY KEY IDENTITY(1,1),
    UsuarioId INT NOT NULL,
    FechaAcceso DATETIME NOT NULL,
    DireccionIP VARCHAR(45) NOT NULL,
    EstadoAcceso VARCHAR(20) NOT NULL -- Ejemplos: 'Exitoso', 'Fallido'
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `AuditarAccesos` que utilice un cursor para revisar los accesos recientes y determinar si son legítimos o sospechosos basándose en ciertos criterios (por ejemplo, múltiples intentos fallidos desde la misma IP).

**Requisitos:**

1. Declarar un cursor que seleccione los últimos 100 registros de la tabla `Accesos`.
2. Para cada acceso:
   - Verificar si hay más de 3 intentos de acceso fallidos desde la misma `DireccionIP` en las últimas 24 horas.
   - Si se detecta una actividad sospechosa, marcar el `EstadoAcceso` como 'Fallido'; de lo contrario, 'Exitoso'.
   - Insertar un registro en la tabla `AuditoriaAccesos` con los detalles y el `EstadoAcceso` determinado.
3. Implementar manejo de transacciones para asegurar que todas las auditorías se registren correctamente.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso de auditoría.


In [ ]:
%%sql
CREATE PROCEDURE AuditarAccesos
AS
BEGIN
      DECLARE @AccesoId INT,
              @UsuarioId INT,
              @FechaAcceso DATETIME,
              @DireccionIP VARCHAR(45),
              @EstadoAcceso VARCHAR(20),
              @IntentosFallidos INT;
      DECLARE cursor_accesos CURSOR FOR
      SELECT TOP 100 AccesoId, UsuarioId, FechaAcceso, DireccionIP
      FROM Accesos
      ORDER BY AccesoId DESC;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_accesos;
        FETCH NEXT FROM cursor_accesos INTO @AccesoId, @UsuarioId, @FechaAcceso, @DireccionIP;
        WHILE @@FETCH_STATUS
        BEGIN
          SELECT @IntentosFallidos = COUNT(*)
          FROM Accesos
          WHERE DireccionIP = @DireccionIP AND FechaAcceso >= DATEADD(HOUR, -24, GETDATE()) AND EstadoAcceso = 'Fallido';
          IF @IntentosFallidos > 3
          BEGIN
            SET @EstadoAcceso = 'Fallido';
          END
          ELSE
          BEGIN
            SET @EstadoAcceso = 'Exitoso';
          END
          INSERT INTO AuditoriaAccesos (UsuarioId, FechaAcceso, DireccionIP, EstadoAcceso)
          VALUES (@UsuarioId, @FechaAcceso, @DireccionIP, @EstadoAcceso);
          FETCH NEXT FROM cursor_accesos INTO @AccesoId, @UsuarioId, @FechaAcceso, @DireccionIP;
        END
        CLOSE cursor_accesos;
        DEALLOCATE cursor_accesos;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;


## Ejercicio 16: Consolidación de Feedback de Clientes

**Enunciado:**

Tienes dos tablas: `FeedbackClientes` y `ResumenFeedback`:

```sql
CREATE TABLE FeedbackClientes (
    FeedbackId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    Comentario VARCHAR(500) NOT NULL,
    Calificacion INT NOT NULL, -- 1 a 5
    FechaFeedback DATE NOT NULL
);

CREATE TABLE ResumenFeedback (
    ResumenId INT PRIMARY KEY IDENTITY(1,1),
    ClienteId INT NOT NULL,
    ComentariosTotales INT NOT NULL,
    CalificacionPromedio DECIMAL(3,2) NOT NULL,
    FechaResumen DATE NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ConsolidarFeedback` que utilice un cursor para procesar los comentarios de los clientes y generar un resumen por cliente.

**Requisitos:**

1. Declarar un cursor que seleccione todos los registros de `FeedbackClientes`.
2. Para cada feedback:
   - Calcular la cantidad total de comentarios y la calificación promedio para cada `ClienteId`.
   - Insertar o actualizar un registro en `ResumenFeedback` con los valores calculados y la fecha actual.
3. Implementar manejo de transacciones para asegurar la consistencia de los datos durante la consolidación.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso.


In [ ]:
%%sql
CREATE PROCEDURE ConsolidarFeedback
AS
BEGIN
      DECLARE @ClienteId INT,
              @ComentariosTotales INT,
              @CalificacionPromedio DECIMAL(3,2),
              @FechaResumen DATE;
      DECLARE cursor_feedback CURSOR FOR
      SELECT ClienteId, COUNT(*) AS ComentariosTotales, AVG(Calificacion) AS CalificacionPromedio, GETDATE() AS FechaResumen
      FROM FeedbackClientes
      GROUP BY ClienteId;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_feedback;
        FETCH NEXT FROM cursor_feedback INTO @ClienteId, @ComentariosTotales, @CalificacionPromedio, @FechaResumen;
        WHILE @@FETCH_STATUS
        BEGIN
          INSERT INTO ResumenFeedback (ClienteId, ComentariosTotales, CalificacionPromedio, FechaResumen)
          VALUES (@ClienteId, @ComentariosTotales, @CalificacionPromedio, @FechaResumen);
          FETCH NEXT FROM cursor_feedback INTO @ClienteId, @ComentariosTotales, @CalificacionPromedio, @FechaResumen;
        END
        CLOSE cursor_feedback;
        DEALLOCATE cursor_feedback;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;

## Ejercicio 17: Monitoreo de Temperaturas en Dispositivos IoT

**Enunciado:**

Tienes una tabla `LecturasTemperatura` que almacena las lecturas de temperatura de diferentes dispositivos IoT:

```sql
CREATE TABLE LecturasTemperatura (
    LecturaId INT PRIMARY KEY IDENTITY(1,1),
    DispositivoId INT NOT NULL,
    Temperatura DECIMAL(5,2) NOT NULL,
    FechaLectura DATETIME NOT NULL
);

CREATE TABLE AlertasTemperatura (
    AlertaId INT PRIMARY KEY IDENTITY(1,1),
    DispositivoId INT NOT NULL,
    Temperatura DECIMAL(5,2) NOT NULL,
    FechaAlerta DATETIME NOT NULL,
    Mensaje VARCHAR(255) NOT NULL
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `MonitorearTemperaturas` que utilice un cursor para revisar las últimas lecturas de temperatura y generar alertas si la temperatura excede ciertos umbrales.

**Requisitos:**

1. Declarar un cursor que seleccione las últimas 100 lecturas de la tabla `LecturasTemperatura`.
2. Para cada lectura:
   - Si la `Temperatura` es mayor a 75°C, generar una alerta con el mensaje "Temperatura Alta".
   - Si la `Temperatura` es menor a 0°C, generar una alerta con el mensaje "Temperatura Baja".
   - Insertar un registro en la tabla `AlertasTemperatura` con los detalles de la alerta.
3. Implementar manejo de transacciones para asegurar que las inserciones de alertas se realicen correctamente.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso de monitoreo.


In [ ]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE MonitorearTemperaturas
AS
BEGIN
      DECLARE @LecturaId INT,
              @DispositivoId INT,
              @Temperatura DECIMAL(5,2),
              @FechaLectura DATETIME,
              @Mensaje VARCHAR(255);
      DECLARE cursor_lecturas CURSOR FOR
      SELECT TOP 100 LecturaId, DispositivoId, Temperatura, FechaLectura
      FROM LecturasTemperatura
      ORDER BY LecturaId DESC;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_lecturas;
        FETCH NEXT FROM cursor_lecturas INTO @LecturaId, @DispositivoId, @Temperatura, @FechaLectura;
        WHILE @@FETCH_STATUS
        BEGIN
          IF @Temperatura > 75
          BEGIN
            SET @Mensaje = 'Temperatura Alta';
          END
          ELSE IF @Temperatura
          BEGIN
            SET @Mensaje = 'Temperatura Baja';
          END
          INSERT INTO AlertasTemperatura (DispositivoId, Temperatura, FechaAlerta, Mensaje)
          VALUES (@DispositivoId, @Temperatura, @FechaLectura, @Mensaje);
          FETCH NEXT FROM cursor_lecturas INTO @LecturaId, @DispositivoId, @Temperatura, @FechaLectura;
        END
        CLOSE cursor_lecturas;
        DEALLOCATE cursor_lecturas;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;

## Ejercicio 18: Actualización de Status de Proyectos Basada en Fechas

**Enunciado:**

Tienes una tabla `Proyectos` que contiene información sobre diferentes proyectos en una empresa:

```sql
CREATE TABLE Proyectos (
    ProyectoId INT PRIMARY KEY IDENTITY(1,1),
    NombreProyecto VARCHAR(100) NOT NULL,
    FechaInicio DATE NOT NULL,
    FechaFin DATE,
    Estado VARCHAR(20) NOT NULL -- Ejemplos: 'En Curso', 'Completado', 'Retrasado'
);
```

**Objetivo:**

Crear un procedimiento almacenado llamado `ActualizarStatusProyectos` que utilice un cursor para revisar cada proyecto y actualizar su estado basado en la fecha actual y la fecha de finalización prevista.

**Requisitos:**

1. Declarar un cursor que seleccione `ProyectoId`, `FechaInicio`, `FechaFin`, y `Estado` de todos los proyectos.
2. Para cada proyecto:
   - Si la `FechaFin` está definida y la fecha actual es mayor que `FechaFin` y el estado no es 'Completado', actualizar el estado a 'Retrasado'.
   - Si la `FechaFin` está definida y la fecha actual es igual o menor que `FechaFin` y el estado es 'Retrasado', actualizar el estado a 'En Curso'.
   - Si el proyecto ya está 'Completado', no realizar cambios.
3. Implementar manejo de transacciones para asegurar que las actualizaciones se realicen correctamente.
4. Manejar adecuadamente la apertura y cierre del cursor, así como posibles errores durante el proceso de actualización.

In [1]:
# Escriba aquí la solución del ejercicio
%%sql
CREATE PROCEDURE ActualizarStatusProyectos
AS
BEGIN
      DECLARE @ProyectoId INT,
              @NombreProyecto VARCHAR(100),
              @FechaInicio DATE,
              @FechaFin DATE,
              @Estado VARCHAR(20);
              DECLARE cursor_proyectos CURSOR FOR
      SELECT ProyectoId, NombreProyecto, FechaInicio, FechaFin, Estado
      FROM Proyectos;
      BEGIN TRY
        BEGIN TRANSACTION;
        OPEN cursor_proyectos;
        FETCH NEXT FROM cursor_proyectos INTO @ProyectoId, @NombreProyecto, @FechaInicio, @FechaFin, @Estado;
        WHILE @@FETCH_STATUS
        BEGIN
          IF @FechaFin IS NOT NULL AND GETDATE() > @FechaFin AND @Estado <> 'Completado'
          BEGIN
            UPDATE Proyectos
            SET Estado = 'Retrasado'
            WHERE ProyectoId = @ProyectoId;
          END
          ELSE IF @FechaFin IS NOT NULL AND
          BEGIN
            UPDATE Proyectos
            SET Estado = 'En Curso'
            WHERE ProyectoId = @ProyectoId;
          END
          FETCH NEXT FROM cursor_proyectos INTO @ProyectoId, @NombreProyecto, @FechaInicio, @FechaFin, @Estado;
        END
        CLOSE cursor_proyectos;
        DEALLOCATE cursor_proyectos;
        COMMIT TRANSACTION;
      END TRY
      BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
      END CATCH
END;


UsageError: Cell magic `%%sql` not found.
